In [1]:
import datetime
import sys
import pymc3 as pm
import theano.tensor as tt
import pickle
import pandas as pd
import numpy as np
import scipy.special
from multiprocessing import cpu_count


sys.path.append("../covid19_inference")
sys.path.append("..")

import data
from causal_covid.data import load_cases, load_infectiability
from causal_covid.utils import get_cps, day_to_week_matrix
from covid19_inference import Cov19Model
from covid19_inference.model import (
    lambda_t_with_sigmoids,
    kernelized_spread_gender,
    week_modulation,
    student_t_likelihood,
    delay_cases,
    uncorrelated_prior_E,
)

begin = datetime.datetime(2021,8,1)
end = datetime.datetime(2021,10,1)
file = "./../data/2022-02-09_16-39-19_young_to_old_cap/scenario_export/observed_infection_data.csv"
cases_df = load_cases(file, begin, end)


vaccination_file = "./../data/2022-02-09_16-39-19_young_to_old_cap/scenario_export/observed_vaccination_data.csv"
waning_file = "./../data/2022-02-09_16-39-19_young_to_old_cap/scenario_export/vaccine_efficacy_waning_data.csv"
population_file = "./../data/2022-02-09_16-39-19_young_to_old_cap/scenario_export/population_data.csv"
observed_U2_file = "./../data/2022-02-02_14-19-24_observed_vac_policy/vaccination_policy/U_2.npy"
observed_U3_file = "./../data/2022-02-02_14-19-24_observed_vac_policy/vaccination_policy/u_3.npy"
scenario_U2_file = "./../data/2022-02-09_16-39-19_young_to_old_cap/vaccination_policy/U_2.npy"
scenario_U3_file = "./../data/2022-02-09_16-39-19_young_to_old_cap/vaccination_policy/u_3.npy"

#pd.read_csv(waning_file)

infectiability = load_infectiability(vaccination_file, population_file, observed_U2_file, observed_U3_file, waning_file, begin, end)

In [2]:
cases_df

,0-29,30-59,60+
2021-08-01,12118.0,8052.0,3326.0
2021-08-08,19348.0,13693.0,4974.0
2021-08-15,28228.0,18120.0,4721.0
2021-08-22,36416.0,20764.0,3755.0
2021-08-29,43083.0,19677.0,2982.0
2021-09-05,37803.0,15010.0,2145.0
2021-09-12,37234.0,14876.0,2185.0
2021-09-19,27093.0,11286.0,1866.0
2021-09-26,16557.0,7082.0,1245.0


In [3]:
infectiability

,0-29,30-59,60+
2021-08-01,0.537637,0.494361,0.314466
2021-08-08,0.536997,0.496784,0.215036
2021-08-15,0.532992,0.424556,0.222917
2021-08-22,0.529001,0.346583,0.230836
2021-08-29,0.475234,0.329432,0.233777
2021-09-05,0.481556,0.352687,0.244757
2021-09-12,0.473987,0.344667,0.247420
2021-09-19,0.472585,0.345070,0.252502
2021-09-26,0.459111,0.316464,0.251993
